In [39]:
%load_ext autoreload
%autoreload 2

from ml.preprocessing import DataPreprocessor

import warnings

import pandas as pd

pd.options.display.max_columns, pd.options.display.max_rows = 5000, 500
pd.options.display.float_format = '{:.3f}'.format
warnings.filterwarnings("ignore")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
data = DataPreprocessor().preprocessing()

In [41]:
data.head()

,uid,fc_imp_chk,fc_time_chk,utmtr,mm_dma,osName,model,hardware,site_id,tag,domain,is_sport_site,is_travel_site,is_weather_site,is_game_site,is_music_site,is_cook_site,is_health_site,is_zodiac_site,year,month,hour,week_day,click
0,00000000-0000-0001-6268-957597321099,0,7,6,517,Android,SM-N960U,Mobile Phone,dotesports.com,vcontent,com,True,False,False,False,False,False,False,False,2021,7,19,Wednesday,False
1,00000000-0000-0001-6268-957946481763,0,7,6,505,Android,Pixel 4a,Mobile Phone,other,vregistration,other,False,False,False,False,False,False,False,False,2021,7,19,Wednesday,False
2,00000000-0000-0001-6268-958052831367,0,7,6,501,iOS,iPhone,Mobile Phone,other,vregistration,other,False,False,False,False,False,False,False,False,2021,7,19,Wednesday,False
3,00000000-0000-0001-6268-958153731939,0,7,6,517,Android,SM-G991U,Mobile Phone,other,vregistration,other,False,False,False,False,False,False,False,False,2021,7,19,Wednesday,False
4,00000000-0000-0001-6268-959263331420,0,7,6,517,Android,SM-N975U,Mobile Phone,other,vregistration,other,False,False,False,False,False,False,False,False,2021,7,19,Wednesday,False


In [42]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9583 entries, 0 to 9582
Data columns (total 24 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   uid              9583 non-null   object
 1   fc_imp_chk       9583 non-null   int64 
 2   fc_time_chk      9583 non-null   int64 
 3   utmtr            9583 non-null   int64 
 4   mm_dma           9583 non-null   int64 
 5   osName           9583 non-null   object
 6   model            9583 non-null   object
 7   hardware         9583 non-null   object
 8   site_id          9583 non-null   object
 9   tag              9583 non-null   object
 10  domain           9583 non-null   object
 11  is_sport_site    9583 non-null   bool  
 12  is_travel_site   9583 non-null   bool  
 13  is_weather_site  9583 non-null   bool  
 14  is_game_site     9583 non-null   bool  
 15  is_music_site    9583 non-null   bool  
 16  is_cook_site     9583 non-null   bool  
 17  is_health_site   9583 non-null   

In [43]:
data['mm_dma'].value_counts()

mm_dma
501    1273
524    1261
517     759
0       656
505     452
511     437
609     246
659     244
518     240
630     227
510     206
544     179
527     165
563     147
515     116
512     115
561     115
504     112
560     112
556     109
529     107
616     106
535      93
691      88
640      86
539      85
622      83
602      83
522      76
567      75
686      73
507      68
573      67
546      60
509      58
882      57
716      52
542      51
551      47
698      46
519      45
718      45
570      44
648      42
520      39
693      38
575      36
503      36
545      35
513      35
566      34
612      34
617      32
675      27
710      25
557      24
642      24
548      21
547      20
588      19
525      18
592      17
746      16
530      15
550      15
623      14
534      13
692      12
673      12
606      12
709      12
656      12
584      11
576      10
671      10
577      10
605       9
632       8
536       7
618       7
628       6
627       6
610      

In [44]:
data['mm_dma'].min()

0

In [45]:
data[data['mm_dma'] == -1]

,uid,fc_imp_chk,fc_time_chk,utmtr,mm_dma,osName,model,hardware,site_id,tag,domain,is_sport_site,is_travel_site,is_weather_site,is_game_site,is_music_site,is_cook_site,is_health_site,is_zodiac_site,year,month,hour,week_day,click


In [46]:
# model

Train

In [3]:
from ml.train_creator import TrainCreator

train, validation = TrainCreator().transform()

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7666 entries, 1992 to 4633
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   uid              7666 non-null   object  
 1   fc_imp_chk       7666 non-null   int64   
 2   fc_time_chk      7666 non-null   int64   
 3   utmtr            7666 non-null   int64   
 4   mm_dma           7666 non-null   int64   
 5   osName           7666 non-null   category
 6   model            7666 non-null   category
 7   hardware         7666 non-null   category
 8   domain           7666 non-null   category
 9   is_sport_site    7666 non-null   bool    
 10  is_travel_site   7666 non-null   bool    
 11  is_weather_site  7666 non-null   bool    
 12  is_game_site     7666 non-null   bool    
 13  is_music_site    7666 non-null   bool    
 14  is_cook_site     7666 non-null   bool    
 15  is_health_site   7666 non-null   bool    
 16  is_zodiac_site   7666 non-null   bool    
 1

In [5]:
train['domain'].value_counts()

domain
com      6027
other    1527
mobi       29
org        15
pl         15
net        12
gr         11
ro         10
uk          9
rs          6
sk          5
Name: count, dtype: int64

LGBM

In [50]:
from ml.click_lgbm import ClickLGBM

In [52]:
ClickLGBM().cross_val_score() # 4460574544493795

0.44617895032691823

In [53]:
feature_importance = ClickLGBM().feature_importance()
feature_importance

[LightGBM] [Warning] bagging_freq is set=8, subsample_freq=0 will be ignored. Current value: bagging_freq=8
[LightGBM] [Warning] lambda_l1 is set=1.26258, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.26258
[LightGBM] [Warning] lambda_l2 is set=9.50238, reg_lambda=0.0 will be ignored. Current value: lambda_l2=9.50238
[LightGBM] [Warning] feature_fraction is set=0.45, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.45
[LightGBM] [Warning] bagging_fraction is set=0.844, subsample=1.0 will be ignored. Current value: bagging_fraction=0.844
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] bagging_freq is set=8, subsample_freq=0 will be ignored. Current value: bagging_freq=8
[LightGBM] [Warning] lambda_l1 is set=1.26258, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.26258
[LightGBM] [Warning] lambda_l2 is set=9.50238, reg_lambda=0.0 will be ignored. Cu

,feature,gain
0,hour,25.068
1,mm_dma,19.148
2,model,13.921
3,utmtr,13.382
4,week_day,11.792
5,fc_imp_chk,5.568
6,month,3.364
7,domain,2.129
8,osName,2.020
9,hardware,1.167


In [54]:
ClickLGBM().save_model()

[LightGBM] [Warning] bagging_freq is set=8, subsample_freq=0 will be ignored. Current value: bagging_freq=8
[LightGBM] [Warning] lambda_l1 is set=1.26258, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.26258
[LightGBM] [Warning] lambda_l2 is set=9.50238, reg_lambda=0.0 will be ignored. Current value: lambda_l2=9.50238
[LightGBM] [Warning] feature_fraction is set=0.45, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.45
[LightGBM] [Warning] bagging_fraction is set=0.844, subsample=1.0 will be ignored. Current value: bagging_fraction=0.844
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] bagging_freq is set=8, subsample_freq=0 will be ignored. Current value: bagging_freq=8
[LightGBM] [Warning] lambda_l1 is set=1.26258, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.26258
[LightGBM] [Warning] lambda_l2 is set=9.50238, reg_lambda=0.0 will be ignored. Cu

In [55]:
validation = ClickLGBM().validation()
validation.head()

[LightGBM] [Warning] lambda_l1 is set=1.26258, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.26258
[LightGBM] [Warning] bagging_freq is set=8, subsample_freq=0 will be ignored. Current value: bagging_freq=8
[LightGBM] [Warning] lambda_l2 is set=9.50238, reg_lambda=0.0 will be ignored. Current value: lambda_l2=9.50238
[LightGBM] [Warning] bagging_fraction is set=0.844, subsample=1.0 will be ignored. Current value: bagging_fraction=0.844
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] feature_fraction is set=0.45, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.45
LogLoss:  0.4538241908344561
Accuracy:  0.8059467918622848


,fc_imp_chk,fc_time_chk,utmtr,mm_dma,osName,model,hardware,domain,is_sport_site,is_travel_site,is_weather_site,is_game_site,is_music_site,is_cook_site,is_health_site,is_zodiac_site,year,month,hour,week_day,click,prediction
0,0,7,0,527,Android,SM-G973U,Mobile Phone,com,False,False,False,False,False,False,False,False,2021,7,1,Monday,False,0.524
1,0,7,4,518,iOS,iPhone,Mobile Phone,com,False,False,False,False,False,False,False,False,2021,8,12,Tuesday,False,0.830
2,0,7,5,539,iOS,iPhone,Mobile Phone,com,False,False,False,False,False,False,False,False,2021,7,16,Friday,True,0.083
3,0,7,5,539,iOS,iPhone,Mobile Phone,other,False,False,False,False,False,False,False,False,2021,7,17,Wednesday,False,0.080
4,0,7,6,505,Android,SM-N975U,Mobile Phone,com,False,False,False,False,False,False,False,False,2021,7,19,Sunday,True,0.722


In [58]:
experiments = ClickLGBM().optuna_optimization(200)

[I 2024-01-11 18:35:39,410] A new study created in memory with name: no-name-86d2d1b7-6293-435d-988a-72265a95d8b0


  0%|          | 0/200 [00:00<?, ?it/s]

[I 2024-01-11 18:36:04,637] Trial 0 finished with value: 0.4509281534157452 and parameters: {'learning_rate': 0.006836, 'feature_fraction': 0.499, 'num_leaves': 15, 'lambda_l1': 2.7071400000000003, 'lambda_l2': 9.28332, 'bagging_fraction': 0.847, 'bagging_freq': 8}. Best is trial 0 with value: 0.4509281534157452.
[I 2024-01-11 18:36:10,533] Trial 1 finished with value: 0.4526883440066973 and parameters: {'learning_rate': 0.029289, 'feature_fraction': 0.45, 'num_leaves': 21, 'lambda_l1': 2.6861200000000003, 'lambda_l2': 4.323410000000001, 'bagging_fraction': 0.847, 'bagging_freq': 190}. Best is trial 0 with value: 0.4509281534157452.
[I 2024-01-11 18:36:28,005] Trial 2 finished with value: 0.45178397965658856 and parameters: {'learning_rate': 0.008861, 'feature_fraction': 0.499, 'num_leaves': 18, 'lambda_l1': 3.4494300000000004, 'lambda_l2': 5.87466, 'bagging_fraction': 0.945, 'bagging_freq': 94}. Best is trial 0 with value: 0.4509281534157452.
[I 2024-01-11 18:36:35,238] Trial 3 finish

Traceback (most recent call last):
  File "/Users/victorialokteva/LLM-test-task/venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/6y/2tqks33d0ts5323j445kpd9m0000gn/T/ipykernel_1609/3833431663.py", line 1, in <module>
    experiments = ClickLGBM().optuna_optimization(200)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/victorialokteva/LLM-test-task/ml/click_lgbm.py", line 87, in optuna_optimization
  File "/Users/victorialokteva/LLM-test-task/venv/lib/python3.11/site-packages/optuna/study/study.py", line 451, in optimize
    _optimize(
  File "/Users/victorialokteva/LLM-test-task/venv/lib/python3.11/site-packages/optuna/study/_optimize.py", line 66, in _optimize
    _optimize_sequential(
  File "/Users/victorialokteva/LLM-test-task/venv/lib/python3.11/site-packages/optuna/study/_optimize.py", line 163, in _optimize_sequential
    frozen_trial = _run_

CatBoost

In [1]:
from ml.click_catboost import ClickCatBoost

/Users/victorialokteva/LLM-test-task/venv/lib/python3.11/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_paths" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [2]:
ClickCatBoost().cross_val_score()

Training on fold [0/5]

bestTest = 0.4375344249
bestIteration = 347

Training on fold [1/5]

bestTest = 0.4199002918
bestIteration = 579

Training on fold [2/5]

bestTest = 0.4451027936
bestIteration = 283

Training on fold [3/5]

bestTest = 0.4460041199
bestIteration = 311

Training on fold [4/5]

bestTest = 0.4347041223
bestIteration = 543

     iterations  test-Logloss-mean  test-Logloss-std  train-Logloss-mean  \
0             0           0.674239          0.001922            0.674204   
1             1           0.659603          0.001575            0.659284   
2             2           0.645687          0.001780            0.645041   
3             3           0.635149          0.003727            0.634517   
4             4           0.622845          0.005035            0.622176   
..          ...                ...               ...                 ...   
675         675           0.438174          0.010804            0.343516   
676         676           0.438176          0.0

0.4382027887487491

In [3]:
ClickCatBoost().save_model()

In [2]:
validation = ClickCatBoost().validation()
validation.head()

LogLoss:  0.44523240985998047
Accuracy:  0.8064684402712572


,index,uid,fc_imp_chk,fc_time_chk,utmtr,mm_dma,osName,model,hardware,domain,...,is_game_site,is_music_site,is_cook_site,is_health_site,is_zodiac_site,year,month,hour,week_day,click
0,3264,00000000-0000-0001-6272-613604941017,0,7,0,527,Android,SM-G973U,Mobile Phone,com,...,False,False,False,False,False,2021,7,1,Monday,False
1,7498,00000000-0000-0001-6279-945387121616,0,7,4,518,iOS,iPhone,Mobile Phone,com,...,False,False,False,False,False,2021,8,12,Tuesday,False
2,5630,00000000-0000-0001-6276-627418591735,0,7,5,539,iOS,iPhone,Mobile Phone,com,...,False,False,False,False,False,2021,7,16,Friday,True
3,4673,00000000-0000-0001-6274-939137071136,0,7,5,539,iOS,iPhone,Mobile Phone,other,...,False,False,False,False,False,2021,7,17,Wednesday,False
4,3147,00000000-0000-0001-6272-400916361907,0,7,6,505,Android,SM-N975U,Mobile Phone,com,...,False,False,False,False,False,2021,7,19,Sunday,True
